*이탤릭체 텍스트*# 🇺Hugging Face Transformers Text Classification Tutorial (Colab)

*Created on: 2025-09-06 11:35:23*

This notebook is designed to run smoothly on Google Colab (T4).

## Learning Objectives
1. 🤗 Load a binary voice-phishing dataset using Hugging Face Datasets
2. 🧠 Fine-tune a Korean text classification model using `klue/roberta-small`
3. 🧩 Understand how to use `AutoModelForSequenceClassification`, `TrainingArguments`, and `Trainer`
4. 🧰 Inspect the Tokenizer and `DataCollatorWithPadding`
5. 📊 Evaluate (Accuracy/F1) after training and practice making predictions

## Resources
- **Model**: `klue/roberta-small`
- **Dataset**: `HyaDoo/ko-voicephishing-binary-classification` (text → binary classification for voice phishing)


## 0. Check runtime environment & install required packages
Install the basic packages to run quickly in a Colab T4 environment.

(If running locally, you can skip GPU-specific checks.)

In [ ]:
!nvidia-smi -L || true
!python -V

In [ ]:
%%capture
!pip install -q transformers datasets accelerate evaluate scikit-learn

## 1. Import libraries & set random seed

Import required libraries and fix random seeds for reproducible training runs.

In [ ]:
import os, random, numpy as np, torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import evaluate

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## 2. Dataset loading and structure check
- Dataset: **`HyaDoo/ko-voicephishing-binary-classification`**
- Main field: `text` (ASR output text) → label (voice-phishing or not)

Split names (train/validation/test) may differ by dataset; the code below robustly handles various cases.

In [ ]:
dataset_name = 'HyaDoo/ko-voicephishing-binary-classification'
raw = load_dataset(dataset_name)
raw

### 2-1. Normalize splits (create if missing) & detect label column
Some public datasets use different label field names like `label`/`labels`/`target`. The utility below auto-detects the label column.

In [ ]:
from datasets import DatasetDict

def pick_label_column(example_keys):
    candidates = ['label', 'labels', 'target', 'y']
    for c in candidates:
        if c in example_keys:
            return c
    return None

# Normalize splits:
# - If the dataset only contains a single 'train' split, create train/test split (80/20) and name the test split 'test'.
# - If 'train' exists alongside 'validation' or 'test', keep as-is.
# - If no 'train' split exists, split the first available split into train/test.
split_names = list(raw.keys())
if 'train' in raw and len(split_names) == 1:
    print("Dataset contains only 'train' split: creating train/test split (80/20). Using 'test' as the evaluation split.")
    tmp = raw['train'].train_test_split(test_size=0.2, seed=seed)
    ds = DatasetDict({'train': tmp['train'], 'test': tmp['test']})
elif 'train' in raw:
    # Dataset already has train (and possibly validation/test)
    # Prefer keeping 'test' if it exists; otherwise keep any existing 'validation' as-is but ensure we have a 'test' key for consistency.
    ds = raw
    if 'test' not in ds and 'validation' in ds:
        # keep existing 'validation' but also make it available under 'test'
        ds['test'] = ds['validation']
else:
    # No explicit 'train' split: split the first available split into train/test
    first_split_name = split_names[0]
    tmp = raw[first_split_name].train_test_split(test_size=0.2, seed=seed)
    ds = DatasetDict({'train': tmp['train'], 'test': tmp['test']})

sample_keys = ds['train'].column_names
label_col = pick_label_column(sample_keys)
text_col = 'text' if 'text' in sample_keys else sample_keys[0]
label_col, text_col, sample_keys

### 2-2. Preview data & label distribution

In [ ]:
from collections import Counter
print("Train sample:")
print(ds['train'][0])

if label_col is not None:
    train_labels = [ex[label_col] for ex in ds['train']]
    print("\nLabel distribution (train):", Counter(train_labels))
else:
    print("[WARNING] Could not detect a label column. Subsequent preprocessing steps may fail.")

## 3. Tokenizer & tokenization
- Model/Tokenizer: `klue/roberta-small`
- Recommended max_length for Colab T4: 128–256 (using 256 here)

In [ ]:
model_name = "klue/roberta-small"   #이 부분을 small ,base, large 모델로 바꿈
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_length = 128
def tokenize_fn(batch):
    return tokenizer(batch[text_col], truncation=True, max_length=max_length)

tokenized = ds.map(
    tokenize_fn,
    batched=True,
    remove_columns=[c for c in ds["train"].column_names if c not in [text_col, label_col]]
)

print(tokenized)



## 4. DataCollatorWithPadding
Use a data collator that automatically pads variable-length batches.
몇 가지 데이터콜레이터가 존재 예를 들어 텍스트클래시피케이션
최종적으로 입력으로 들어갈 때는 텐서가 들어가야하는데 텐서로 바꾸는 마지막 처리(모델로 들어가기 전에) + 패딩 처리를 시행


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

## 5. Model loading (AutoModelForSequenceClassification)
- Set `num_labels=2` for binary classification
- CUDA will be used automatically if a GPU is available

In [ ]:
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
).to(device)
model.config.id2label = {0: 'non_phishing', 1: 'phishing'}
model.config.label2id = {'non_phishing': 0, 'phishing': 1}
model

## 6. Define evaluation metrics
- Use `evaluate` and `scikit-learn` to compute Accuracy, Precision, Recall, and F1

In [ ]:
import numpy as np
accuracy = evaluate.load('accuracy')
precision = evaluate.load('precision')
recall = evaluate.load('recall')
f1 = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy.compute(references=labels, predictions=preds)['accuracy'],
        'precision': precision.compute(references=labels, predictions=preds, average='binary')['precision'],
        'recall': recall.compute(references=labels, predictions=preds, average='binary')['recall'],
        'f1': f1.compute(references=labels, predictions=preds, average='binary')['f1'],
    }

훈련을 시킬 시 트레이닝 할 때 computer metrix를 구현.. 싸이킷런(ppt 예시) 말고 자체 모델을 사용함. 관련 코드를 허브에서 내려받음

## 7. TrainingArguments
- Configure batch size and epochs suitable for Colab T4 (adjust as needed)
- Use FP16 for speed and memory optimization
- Set checkpoint/logging/evaluation frequencies

어디에 체크 포인트를 저장할건가
가장 좋은 모델을 save해서 저장 평가는 metrix f1으로 기준을 잡고 evaluation이 큰 값을 이용할겅

In [ ]:
output_dir = 'outputs-roberta-voicephishing'
args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to=['none'],
    seed=seed,
)
args

## 8. Create Trainer and run training


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized.get('test', tokenized['train'].select(range(100))),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

## 9. Evaluation and prediction testing

In [ ]:
metrics = trainer.evaluate()
metrics

In [ ]:
def predict_texts(texts):
    # Use the trained model object directly
    enc = tokenizer(texts, truncation=True, max_length=max_length, return_tensors='pt', padding=True).to(device)
    with torch.no_grad():
        outputs = model(**enc)
        # Access the logits from the first element of the tuple output
        logits = outputs[0] #가장 큰 값이 나옴
    preds = torch.argmax(logits, dim=-1).cpu().numpy().tolist()
    probs = torch.softmax(logits, dim=-1).cpu().numpy()
    mapped = [model.config.id2label[p] for p in preds]
    return list(zip(texts, mapped, probs.tolist()))

samples = [
    "고객님 대출 한도 상향되어 안내드립니다. 대출 실행을 위해 본인인증 필요합니다.",
    "안녕하세요, 내일 오후 3시에 미팅 가능하신가요?",
]
predict_texts(samples) #확률 값까지 출력 가능

## 10. Model saving and loading
Show how to save the trained weights and tokenizer to disk and reload them.

In [ ]:
from transformers import AutoModelForSequenceClassification

SIZE = "small"  # small / base / large 중 선택
save_dir = f"saved-roberta-{SIZE}-voicephishing"

trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
trainer.model.config.save_pretrained(save_dir)

print('Saved to', save_dir)
reloaded = AutoModelForSequenceClassification.from_pretrained(
    save_dir, local_files_only=True
).to(device)
reloaded


## 11. (Optional) Confusion matrix & classification report
Print a simple performance report on the validation set.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Use the 'test' split for evaluation/reporting
eval_ds = tokenized.get('test', tokenized['train'].select(range(200)))
pred = trainer.predict(eval_ds)
y_true = pred.label_ids
y_pred = pred.predictions.argmax(axis=-1)
print(classification_report(y_true, y_pred, target_names=['non_phishing','phishing']))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

## Tips
- Adjust batch size / epochs / learning rate according to Colab session and dataset size.
- If training is slow, reduce `max_length` to 128 or lower the batch size.
- `report_to=['none']` disables external logging like WandB.
- Public datasets can be loaded without local tokens.


## 12. TODO


- Write code that loads the model trained with a pipeline, takes a sentence as input, and immediately checks the classification result.
- Replace the model with klue/roberta-base and klue/roberta-large, run them, and investigate how each of these models differs from klue/roberta-small.

#13. TODO- homework

In [ ]:
# pip install -U transformers torch accelerate
import os, time, torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from transformers.modeling_outputs import SequenceClassifierOutput

ID2LABEL = {0: "non_phishing", 1: "phishing"}
LABEL2ID = {"non_phishing": 0, "phishing": 1}
MAX_LEN  = 128
POSTPROC = "softmax"
THRESH   = 0.5

SAVE_DIRS = {
    "klue/roberta-small": "saved-roberta-small-voicephishing",
    "klue/roberta-base":  "saved-roberta-base-voicephishing",
    "klue/roberta-large": "saved-roberta-large-voicephishing",
}

def _is_local_model_dir(path: str) -> bool:
    return os.path.isdir(path) and os.path.isfile(os.path.join(path, "config.json"))

def _ensure_label_maps_saved(dirpath: str):
    m = AutoModelForSequenceClassification.from_pretrained(dirpath, local_files_only=True)
    need = False
    if not isinstance(getattr(m.config, "id2label", None), dict) or len(m.config.id2label) != m.config.num_labels:
        m.config.id2label = ID2LABEL; need = True
    if not isinstance(getattr(m.config, "label2id", None), dict) or set(m.config.label2id.keys()) != set(LABEL2ID.keys()):
        m.config.label2id = LABEL2ID; need = True
    if need:
        m.config.save_pretrained(dirpath)
    return need

def _build_pipeline_local(dirpath: str):
    tok = AutoTokenizer.from_pretrained(dirpath, local_files_only=True)
    mdl = AutoModelForSequenceClassification.from_pretrained(dirpath, local_files_only=True)

    mdl.config.return_dict = True
    mdl.config.id2label = getattr(mdl.config, "id2label", ID2LABEL) or ID2LABEL
    mdl.config.label2id = getattr(mdl.config, "label2id", LABEL2ID) or LABEL2ID

    orig_forward = mdl.forward
    def forward(*args, **kwargs):
        kwargs["return_dict"] = True
        out = orig_forward(*args, **kwargs)
        if isinstance(out, tuple):
            return SequenceClassifierOutput(logits=out[0])
        return out
    mdl.forward = forward

    device = 0 if torch.cuda.is_available() else -1
    clf = pipeline(
        "text-classification",
        model=mdl,
        tokenizer=tok,
        device=device,
        framework="pt",
        function_to_apply=("sigmoid" if POSTPROC == "sigmoid" else "softmax"),
        return_all_scores=(POSTPROC == "sigmoid"),
    )
    print(f"[DEBUG] loaded: {dirpath} | id2label={mdl.config.id2label}")
    return clf

def load_all_saved_models():
    models = {}
    for mid, path in SAVE_DIRS.items():
        if not _is_local_model_dir(path):
            print(f"[SKIP] {mid}: {path} 없음 (config.json 미존재)")
            continue
        if _ensure_label_maps_saved(path):
            print(f"[FIX] 라벨 매핑 저장 완료 → {path}/config.json")
        models[mid] = _build_pipeline_local(path)
    if not models:
        raise RuntimeError("로컬 체크포인트를 하나도 찾지 못했습니다. SAVE_DIRS 경로를 확인하세요.")
    return models

def test_single_model(clf, name, text):
    t0 = time.time()
    out = clf(text, truncation=True, max_length=MAX_LEN)
    dt = time.time() - t0

    if POSTPROC == "softmax":
        res = out[0]  # {'label','score'}
    else:
        scores = {d["label"]: float(d["score"]) for d in out[0]}
        pos = scores.get("phishing", None)
        assert pos is not None, f"'phishing' 점수 없음: keys={list(scores.keys())}"
        pred = "phishing" if pos >= THRESH else "non_phishing"
        res = {"label": pred, "score": pos}

    print(f"[{name}] {res['label']} (score={res['score']:.4f})  {dt:.4f}s")
    return res, dt

def compare_summary(results, times):
    print("\n=== 비교 요약 ===")
    print("1) 라벨/스코어")
    for name, r in results.items():
        print(f" - {name:20}: {r['label']} ({r['score']:.4f})")
    print("\n2) 처리시간(오름차순)")
    for name, t in sorted(times.items(), key=lambda x: x[1]):
        print(f" - {name:20}: {t:.4f}s")

def main():
    print("[cwd]", os.getcwd())
    print("[ls ]", os.listdir("."))
    for mid, p in SAVE_DIRS.items():
        print(f"[check] {mid} -> {p} : {_is_local_model_dir(p)}")

    models = load_all_saved_models()

    samples = [
        "고객님 대출 한도 상향되어 안내드립니다. 대출 실행을 위해 본인인증 필요합니다.",
        "안녕하세요, 내일 오후 3시에 미팅 가능하신가요?",
    ]

    for i, text in enumerate(samples, 1):
        print("\n" + "-"*60)
        print(f"샘플 {i}: {text}")
        res, tm = {}, {}
        for mid, clf in models.items():
            r, t = test_single_model(clf, mid, text)
            res[mid], tm[mid] = r, t
        compare_summary(res, tm)

if __name__ == "__main__":
    main()


- 세 모델 모두 **non_phishing**으로 분류.
- **스코어**: small(0.99) < base(0.992~0.999) < large(0.999)  
- **처리 시간**: 일반적으로 *small < base < large*, 단 일부 샘플에서는 Base가 더 빠른 경우도 있었다.
- **결론**: 모델 크기가 커질수록 스코어가 올라가고 처리 시간도 증가한다, 따라서
모델 선택 시 적절한 고려가 필요하다.